In [2]:
import smbus2
import time
from IPython.display import clear_output
import datetime

In [3]:
addr_sht = 0x44
addr_switch = 0x70
addr_sdcard = 0x71
addr_charger = 0x6a
addr_gauge = 0x55
addr_rtc = 0x51
addr_eeprom = 0x50
addr_eepromsn = 0x58
addr_altimet = 0x77

addresses = {
    'sht': addr_sht, 
    'switch': addr_switch, 
    'sdcard': addr_sdcard,
    'charger': addr_charger,
    'gauge': addr_gauge,
    'rtc': addr_rtc,
    'eeprom': addr_eeprom,
    'eepromsn': addr_eepromsn, 
#    'altimet': addr_altimet
    }

i2c_names = {
    'sht': "Hygrometer",
    'switch': "I2C master switch",
    'sdcard': "Memory controller",
    'charger': "Battery chager",
    'gauge': 'Battery meter',
    'rtc': 'Real-time',
    'eeprom': 'EEPROM memory',
    'eepromsn': 'Serial number',
    'altimet': 'Barometer'
}

In [4]:
def read_2(c, addr, reg):
    d = c.read_i2c_block_data(addr, reg, 2)
    #print([bin(x) for x in d])

    d = d[0]| d[1] << 8
    #print(bin(d))
    return d

def crop(number, start=0, end=16):
    # Vytvoření masky s jedničkami na pozicích od start do end
    mask = ((1 << (end - start + 1)) - 1) << start
    
    # Použití masky pro vybrání příslušných bitů
    result = (number & mask) >> start
    
    return result

def comp(data, length=16):

    if data & (1 << (length-1)):
        data = data ^ ((1 << length) - 1)
        data = -(data + 1)

    return data



In [5]:
try:
    c.close()
except Exception as e:
    print("na")

na


In [6]:
def con():
    return smbus2.SMBus(8)

c = con()

# I2C switch, switch internal I2C bus into USB (FT260) IO

In [7]:
c=con()

In [8]:
c.write_byte_data(addr_switch, 0x01, 0x03) # nastavit I2C switch do USB

OSError: [Errno 5] Input/output error

In [76]:
hex(c.read_byte(addr_switch, 0x01))

'0xb'

In [77]:
import sys
def scan_i2c_bus(_bus):
  devices = []
  sys.stdout.write ('   ')
  for _address in range(16):
    sys.stdout.write (' %2x' % _address) # Print header
  
  for _address in range(128):
    if not _address % 16:
      sys.stdout.write ('\n%02x:' % _address) # Print address
    if 2 < _address < 120 : # Skip reserved addresses
      try:
        _bus.read_byte(_address)
        sys.stdout.write (' %02x' % _address) # Device address
        devices.append(_address)
      except:
        sys.stdout.write (' --') # No device detected
    else:
      sys.stdout.write ('   ') # Reserved

  sys.stdout.write ('\n')
  return devices

devices = scan_i2c_bus(c)
devices = set(devices)

print("\n\n")
for an in addresses:
  print(f"(0x{addresses[an]:02x}) - {an}: {addresses[an] in devices}" )
  #print(an, addressesav, av in devices)

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- -- -- -- -- -- -- -- --
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
40: -- -- -- --

 -- -- -- -- -- -- -- -- -- -- -- --
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
70: -- -- -- -- -- -- -- --                        



(0x44) - sht: False
(0x70) - switch: False
(0x71) - sdcard: False
(0x6a) - charger: False
(0x55) - gauge: False
(0x51) - rtc: False
(0x50) - eeprom: False
(0x58) - eepromsn: False


# Nabíječka 

In [78]:
# Vypnout bateriovy modul
if 0:
    c.write_byte_data(addr_charger, 0x18, 0x0a)

In [79]:
if 0:

    charge_current_ma = 500

    c.write_byte_data(addr_charger, 0x10, 0x1f << 3)

    c.write_byte_data(addr_charger, 0x02, int(charge_current_ma/40)<<5)
    c.write_byte_data(addr_charger, 0x14, 0b00100110)
    c.write_byte_data(addr_charger, 0x15, 0b00011101)
    c.write_byte_data(addr_charger, 0x16, 0b10100000)
    c.write_byte_data(addr_charger, 0x17, 0b01010110)
    c.write_byte_data(addr_charger, 0x18, 0b00000000)
    c.write_byte_data(addr_charger, 0x19, 0b00000001)

    # NTC
    c.write_byte_data(addr_charger, 0x1a, 0b10111111)

    # ADC
    c.write_byte_data(addr_charger, 0x26, 0b10001100)
    
c.write_byte_data(addr_charger, 0x26, 0b10001100)

OSError: [Errno 5] Input/output error

In [ ]:

while True:
    # Zapnout interni ADC prevodnik
    #print(bin(c.read_byte_data(addr_charger, 0x26)))
    #c.write_byte_data(addr_charger, 0x26, 0b10001100)
    
    clear_output()
    print("IBUS ADC", comp(crop(read_2(c, addr_charger, 0x28), 1, 16), 15) * 2, " mA")
    #print(" ")

    print("IBAT ADC", comp(crop(read_2(c, addr_charger, 0x2A), 2, 16), 14) * 4, " mA")
    #print(" ")

    print("VBUS ADC", crop(read_2(c, addr_charger, 0x2C), 2, 15) * 3.97, " mV")
    #print(" ")

    print("VPMID ADC", crop(read_2(c, addr_charger, 0x2E), 2, 15) * 3.97, " mV")
    #print(" ")

    charger_vbat = crop(read_2(c, addr_charger, 0x30), 1, 13) * 1.99
    print("VBAT ADC", charger_vbat, " mV")
    #print(" ")

    print("VSYS ADC", crop(read_2(c, addr_charger, 0x32), 1, 13) * 1.99, " mV")
    #print(" ")

    #print("TS ADC", crop(read_2(c, addr_charger, 0x34), 0, 12) * 0.0961, " %")
    #print(" ")

    #print("TDIE ADC", crop(read_2(c, addr_charger, 0x36), 0, 12) * 0.5 - 40, " C")
    #print(" ")

    time.sleep(1)
    break


IBUS ADC 30  mA
IBAT ADC 0  mA
VBUS ADC 5101.45  mV
VPMID ADC 5093.51  mV
VBAT ADC 73.63  mV
VSYS ADC 3745.18  mV


In [ ]:
# Nastavit timery
c.write_byte_data(addr_charger, 0x15, 0b00001001)

In [ ]:
d = c.read_byte_data(addr_charger, 0x1d)
print("Charger status 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1e)
print("Charger status 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x1f)
print("Fault reg 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x20)
print("Charger flag 0: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x21)
print("Charger flag 1: 0b{0:08b}".format(d))

d = c.read_byte_data(addr_charger, 0x22)
print("Fault flag 0: 0b{0:08b}".format(d))

Charger status 0: 0b00010000
Charger status 1: 0b00001100
Fault reg 0: 0b00000001
Charger flag 0: 0b00010001
Charger flag 1: 0b00001001
Fault flag 0: 0b00000001


In [ ]:
voltage = c.read_i2c_block_data(0x55,0x08,2)
voltage = (voltage[1] << 8) | voltage[0]
print(f"Voltage {voltage} mV")

current = c.read_i2c_block_data(0x55,0x0a,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA (avg)")

current = c.read_i2c_block_data(0x55,0x10,2)
current = (current[1] << 8) | current[0]
print(f"Current {current} mA")


d = c.read_word_data(0x55,0x04)
print(f"RemainingCapacity {d} mAh")


d = c.read_word_data(0x55,0x06)
print(f"Full Capacity {d} mAh")

d = c.read_word_data(0x55,0x0c)
print(f"Temp {d/10-273} C")


d = c.read_byte(0x55,0x02)
print(f"State: {d} %")


#d = c.read_ (0x55,0x04,2)
#d = (d[1] << 8) | d[0]
#print(f"CRemainingCapacity {d} mAh")

OSError: [Errno 5] Input/output error

In [ ]:
import struct 
bus = c
address = 0x55

e=bus.read_i2c_block_data(address,0x0a,2)
(full_cap,)=struct.unpack('H', bytearray(e)[0:2])
print("Full Available Capacity is",full_cap, 'mA')

#Trying to write the nominal capacity
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)
bus.write_byte_data(address,0x00,0x00)
bus.write_byte_data(address,0x01,0x80)

bus.write_byte_data(address,0x00,0x13)
bus.write_byte_data(address,0x01,0x00)

#bus.write_byte_data(address,0x61,0x00)
#bus.write_byte_data(address,0x3e,0x52)
#bus.write_byte_data(address,0x3f,0x00)

bus.write_byte_data(address,0x4a,0x04) #writing new capacity
bus.write_byte_data(address,0x4b,0xb0) #writing new capacity

time.sleep(1)

#bus.write_byte_data(address,0x60,0x1f) #trying to write on BlockDataChecksum() 

time.sleep(1)

bus.write_byte_data(address,0x00,0x42)
bus.write_byte_data(address,0x01,0x00)

bus.write_byte_data(address,0x00,0x20)
bus.write_byte_data(address,0x01,0x00)

f=bus.read_i2c_block_data(address,0x3c,2) #address for design capacity
(des_cap,)=struct.unpack('H', bytearray(f)[0:2])
print(des_cap)


OSError: [Errno 5] Input/output error

EEPROM + Serial number...

In [ ]:
def read_serial_number(address):
    serial_number = []
    c.write_byte_data(address, 0x08, 0x00)
    for _ in range(16):
        serial_byte = c.read_byte(address)
        serial_number.append(serial_byte)
    
    result_number = 0
    for b in serial_number:
        result_number = (result_number << 8) | b

    return result_number

detector_serial_number = read_serial_number(addr_eepromsn)
print( hex(detector_serial_number) )


0x1290c00806a200919049a000a0000060


In [ ]:

# Příkazy pro čtení teploty a vlhkosti
temperature_cmd = [0x24, 0x00]  # Příkaz pro čtení teploty v režimu High Precision
humidity_cmd = [0x24, 0x16]     # Příkaz pro čtení vlhkosti v režimu High Precision
serial_number_cmd = [0x37, 0x80]

# Funkce pro čtení dat ze senzoru
def sht_read_sensor_data(cmd):
    c.write_i2c_block_data(addr_sht, cmd[0], [cmd[1]])
    time.sleep(0.1)  # Počkejte na dokončení měření (0.1 sekundy)
    data = c.read_i2c_block_data(addr_sht, 0, 6)
    raw_temperature = (data[0] << 8) + data[1]
    raw_humidity = (data[3] << 8) + data[4]
    temperature = -45 + 175 * (raw_temperature / 65535.0)  # Výpočet teploty
    humidity = 100 * (raw_humidity / 65535.0)             # Výpočet vlhkosti
    return temperature, humidity


def sht_read_sn(cmd):
    c.write_i2c_block_data(addr_sht, cmd[0], [cmd[1]])
    data = c.read_i2c_block_data(addr_sht, 0, 6)
    print(data)
    serial_number = (data[0] << 24) | (data[1] << 16) | (data[3] << 8) | data[4]
    return serial_number

# Čtení teploty a vlhkosti ze senzoru
temperature, humidity = sht_read_sensor_data(temperature_cmd)
print("Teplota: {:.2f} °C".format(temperature))
print("Vlhkost: {:.2f} %".format(humidity))

sht_serial_number = sht_read_sn(serial_number_cmd)
print("Sériové číslo: 0x{:x}".format(sht_serial_number))


Teplota: 22.62 °C
Vlhkost: 47.95 %
[0, 83, 172, 187, 134, 214]
Sériové číslo: 0x53bb86


# BAROMETER


In [ ]:
import time
bus = c
#
## This writes the command to the Sensor to calculate the digital value of pressure.  (This is a raw value and will
#       be used in a calculation later on in the script to get the actual real pressure.)
bus.write_byte(0x77, 0x48)
time.sleep(0.05)

## This command reads the data that we just calculated on the sensor
##  I had to use the read_i2c_block_data rather than the read_word or read_byte because the values being returned by the
#       sensor were 24 bit and was being truncated by the other read methods.  Reading the whole data block was the only way
#       that I knew to get all of the data.
D1 = bus.read_i2c_block_data(0x77, 0x00, 4)  
time.sleep(0.05)


## This is much like above only it is getting the raw temperature.
bus.write_byte(0x77, 0x58)
time.sleep(0.05)
D2 = bus.read_i2c_block_data(0x77, 0x00, 4)
time.sleep(0.05)


## The data being read in blocks are in an array in 8 bit pieces, so we need to convert the first 24 bits into decimal,
#       which is what this statement does.
D1 = D1[0] * 65536 + D1[1] * 256.0 + D1[2]
D2 = D2[0] * 65536 + D2[1] * 256.0 + D2[2]


## The MS6511 Sensor stores 6 values in the EPROM memory that we need in order to calculate the actual temperature and pressure
## These values are calculated/stored at the factory when the sensor is calibrated.
##      I probably could have used the read word function instead of the whole block, but I wanted to keep things consistent.
C1 = bus.read_i2c_block_data(0x77, 0xA2, 2) #Pressure Sensitivity
time.sleep(0.05)
C2 = bus.read_i2c_block_data(0x77, 0xA4,2) #Pressure Offset
time.sleep(0.05)
C3 = bus.read_i2c_block_data(0x77, 0xA6,2) #Temperature coefficient of pressure sensitivity
time.sleep(0.05)
C4 = bus.read_i2c_block_data(0x77, 0xA8,2) #Temperature coefficient of pressure offset
time.sleep(0.05)
C5 = bus.read_i2c_block_data(0x77, 0xAA,2) #Reference temperature
time.sleep(0.05)
C6 = bus.read_i2c_block_data(0x77, 0xAC,2) #Temperature coefficient of the temperature
time.sleep(0.05)

## Again here we are converting the 2 8bit packages into a single decimal
C1 = C1[0] * 256.0 + C1[1]
C2 = C2[0] * 256.0 + C2[1]
C3 = C3[0] * 256.0 + C3[1]
C4 = C4[0] * 256.0 + C4[1]
C5 = C5[0] * 256.0 + C5[1]
C6 = C6[0] * 256.0 + C6[1]

## These are the calculations provided in the datasheet for the sensor.
## TEMP is the actual temperature reported in Centigrade.  It is reported to the 1/100ths, so it needs to be divided by 100 to be used by us.
dT = D2 - C5 * 2**8
TEMP = 2000 + dT * C6 / 2**23

alt_temp = TEMP

## I also included Fahrenheit for the rest of us normal people
TEMP_F = TEMP/100.0 * 9.0/5 + 32

print ("Temperature: ", TEMP/100.0)
print ("Temperature F: ", round(TEMP_F, 2))

## These calculations are all used to produce the final pressure value (just as before the Pressure value needs to be divided by 100 to shift the decimal place where it belongs.)
OFF = C2 * 2**16 + (C4 * dT) / 2**7
SENS = C1 * 2**15 + (C3 * dT) / 2**8
P = (D1 * SENS / 2**21 - OFF) / 2**15

alt_press = P/100.0

print ("Pressure: ", P/100.0)

## Now one curious thing I found was that the sensor needs to be adjusted to your current eleivation.  I couldn't find an automatic way to do it, but I did find
#       this website with a chart. http://www.novalynx.com/manuals/bp-elevation-correction-tables.pdf
#   Just replace the 55.5 below with whatever elevation is approapriate for you.
print ("Pressure Adjusted: ", round(P/100.0 + 55.5, 2))

Temperature:  22.57095835208893
Temperature F:  72.63
Pressure:  978.6669030237314
Pressure Adjusted:  1034.17



# RTC

In [ ]:
import datetime

In [ ]:
def get_time():
    r01 = c.read_byte_data(addr_rtc, 0x01)
    r02 = c.read_byte_data(addr_rtc, 0x02)
    r03 = c.read_byte_data(addr_rtc, 0x03)
    r04 = c.read_byte_data(addr_rtc, 0x04)
    r06 = c.read_byte_data(addr_rtc, 0x06)
    r07 = c.read_byte_data(addr_rtc, 0x07)

    sec = ((r01 >> 4) & 0b111) * 10 + (r01 & 0b1111)
    minu= ((r02 >> 4) & 0b111) * 10 + (r02 & 0b1111)
    hour= ((r03 >> 4) & 0b11) * 10 + (r03 & 0b1111)
    day = ((r04 >> 4) & 0b11) * 10 + (r04 & 0b1111)
    mon = ((r06 >> 4) & 0b1) * 10 + (r06 & 0b1111)
    year= ((r07 >> 4) & 0b1111) * 10 + (r07 & 0b1111)+2000

    print(year, mon, day, hour, minu, sec)        
    return datetime.datetime(year, mon, day, hour, minu, sec)
    
def set_time(time):
    c.write_byte_data(addr_rtc, 0x01, (time.second % 10) | int(time.second/10)<<4 )
    c.write_byte_data(addr_rtc, 0x02, (time.minute % 10) | int(time.minute/10)<<4 )
    c.write_byte_data(addr_rtc, 0x03, (time.hour % 10) | int(time.hour/10)<<4 )
    c.write_byte_data(addr_rtc, 0x04, (time.day % 10) | int(time.day/10)<<4 )
    c.write_byte_data(addr_rtc, 0x06, (time.month % 10) | int(time.month/10)<<4 )
    c.write_byte_data(addr_rtc, 0x07, ((time.year-2000) % 10) | int((time.year-2000)/10)<<4 )

In [ ]:
actual_time = datetime.datetime.utcnow()
detector_time = get_time()
print("Actual time", actual_time)
print("Detector time", detector_time)
print("Difference", detector_time-actual_time)

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
actual_time = datetime.datetime.utcnow()
set_time(actual_time)

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

In [ ]:
actual_time.second

43

In [ ]:

time = actual_time

In [ ]:
bin( (time.second % 10)<<4)

'0b110000'

In [ ]:
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
from datetime import datetime
from IPython.display import display, FileLink

# Funkce pro vytvoření a zobrazení PDF reportu s logem a názvem firmy
def create_and_display_pdf_report():
    # Název zařízení
    device_name = "BATDATUNIT01B-EXT"

    # Čas a datum vytvoření
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    # Název firmy
    company_name = "Universal Scientific Technologies s.r.o."  # Nahraďte názvem vaší firmy
    logo_path = "Logo_UST.png"

    # Vytvoření PDF obsahu
    filename = f"batdatunit01a_report_0x{detector_serial_number:0x}.pdf"
    pdf = canvas.Canvas(filename, pagesize=A4)
    pdf.setFont("Helvetica", 12)

    # Přidání loga a názvu firmy do hlavičky
    pdf.drawImage(logo_path, 80, 720, width=100, preserveAspectRatio=1)
    pdf.drawString(280, 765, company_name)
    pdf.drawString(280, 750, "www.ust.cz, support@ust.cz")

    pdf.drawString(80, 690, "Test report for " + device_name + f"    SN: 0x{detector_serial_number:0x}")
    pdf.drawString(80, 675, "Generated on " + current_time)
    
    pdf.drawString(80, 640, "I2C Devices:")
    y = 620


    #print(f"(0x{addresses[an]:02x}) - {an}: {addresses[an] in devices}" )
    for idev in addresses:
        addr = addresses[idev]
        pdf.drawString(100, y, f"{i2c_names.get(idev, idev)}")
        pdf.drawString(230, y, f"0x{addr:02x}")
        pdf.drawString(280, y, "Pass" if (addr in devices) else "Failed" )
        y -= 20

    pdf.drawString(80, 370, "Serial Numbers:")
    pdf.drawString(100, 350, f"Detector Serial Number:")
    pdf.drawString(280, 350, f"0x{detector_serial_number:0x}")
    pdf.drawString(100, 330, f"SHT3x Serial Number:")
    pdf.drawString(280, 330, f"0x{sht_serial_number:0x}")
    #pdf.drawString(70, 310,  "Current Temperature: " + current_temperature)

    pdf.drawString(80, 300, "Measured values")
    pdf.drawString(100, 280, f"Hygrometer:  T: {temperature:.2f} C, H: {humidity:.2f} %  ... Pass")
    pdf.drawString(100, 260, f"Barometer:  T: {alt_temp/100.0:.2f} C, P: {alt_press:.2f} Pa  ... Pass")
    pdf.drawString(100, 240, f"Charger:  Vbat: {charger_vbat/1000:.1f} V  ... Pass")
    detector_time = get_time()
    pdf.drawString(100, 220, f"RTC time: actual: {detector_time}")

    # Uložení PDF
    pdf.save()

    # Zobrazit PDF v Jupyter Notebooku
    display(FileLink(filename))

# Volání funkce pro vytvoření a zobrazení PDF reportu
create_and_display_pdf_report()


2023 12 11 22 45 37


/home/roman/repos/AIRDOS04/sw/batdatunit01a_report_0x1290c00806a200919049a000a0000060.pdf